# Introduction of new Python Package/Tool
---
Installation notes and any related comments:

must have pydantic-settings, numpy, pandas, ydata-profiling, seaborn

### install commands
pip install pydantic-settings
pip install numpy
pip install pandas
pip install ydata-profiling
pip install seaborn

## word count

In [ ]:
import nbformat

# Load your notebook
with open('C:/Users/Angus/programming in python y2/Group-work/Python-Group-Work/CS2PP22_CW2 Notebook.ipynb', 'r', encoding='utf-8') as notebook_file:
    notebook = nbformat.read(notebook_file, as_version=4)

# Function to count words in a markdown cell
def count_words_in_markdown(cell):
    words = cell['source'].split()
    return len(words)

# Calculate the total word count for markdown cells
word_count = sum(count_words_in_markdown(cell) for cell in notebook.cells if cell.cell_type == 'markdown')

print(f"The notebook has approximately {word_count} words in markdown cells.")

#### Module Code: CS2PP22
#### Assignment report Title: Group Python Work Spring Term
#### Date (when the work completed): 13/03/2024
#### Actual hrs spent for the assignment: 11

In [ ]:
# All import statements 
# Packages Explored in Module
import pandas as pd 
import numpy
import matplotlib.pyplot as plt
import os
import seaborn as sns

# New Tool for This Project
import ydata_profiling as pp


%matplotlib inline
sns.set_style("whitegrid")

# The Evolution of Musical Attributes and Their Impact on the Popularity of Songs in the Digital Era

### Problem Statement:
In the age of digital music, the success of a song is often gauged by its popularity on streaming platforms, which can be influenced by various musical attributes. Understanding how different attributes correlate with popularity can provide insights into listener preferences and inform decisions in the music production and marketing processes.

### Objective:

To analyze the relationship between various song attributes such as genre, tempo, and explicit content and their popularity scores.
To identify trends in the evolution of musical attributes over time and how they relate to shifts in listener engagement and popularity.
To determine if there are specific patterns or characteristics that are common among highly popular songs.

### Hypotheses:

Songs with certain attributes, like a higher tempo or inclusion of explicit content, are more likely to achieve high popularity scores.
Changes in listener preferences over time will reflect in the prominence of different musical attributes within the most popular songs.
There may be a pattern of convergence or divergence in song attributes within top-charting hits compared to songs with lower popularity scores.

### Research Questions:

What song attributes most strongly correlate with high popularity scores within the dataset?
How have the common attributes of popular songs changed over the past decade?
Can we predict the potential popularity of a song based on its attributes without considering its audio features?

### Significance:
The project's findings can offer valuable insights for artists, producers, and record labels into the types of song attributes that resonate with listeners in the current music landscape. Additionally, these insights could be used to develop more targeted music recommendation systems that align with evolving listener preferences.

# 2. Collect Data
---

dataset is MusicOSet

In [ ]:
# Get the current working directory
cwd = os.getcwd()

# Construct the path to the "Data" folder
data_folder = os.path.join(cwd, "Data")

# Initialize an empty list to store the data frames
data_frames = {}

# Loop through all files in the "Data" folder
for file in os.listdir(data_folder):
    # Check if the file is a CSV
    if file.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(data_folder, file)
        
        # Read the Excel file into a DataFrame
        df = pd.read_csv(file_path, delimiter='\t')
        
        # Append the DataFrame to the list\n",
        data_frames[file] = df

# Concatenate all the data frames into a single DataFrame
#data = pd.concat(data_frames, ignore_index=True)

data_frames['songs.csv'].columns = data_frames['songs.csv'].columns.map(lambda x : 'song_'+x if 'song_' not in x else x)
data_frames['albums.csv'].columns = data_frames['albums.csv'].columns.map(lambda x : 'album_'+x if 'album_' not in x else x)
data_frames['artists.csv'].columns = data_frames['artists.csv'].columns.map(lambda x : 'artist_'+x if 'artist_' not in x else x)
data_frames['acoustic_features.csv'].columns = data_frames['acoustic_features.csv'].columns.map(lambda x : 'song_'+x if 'song_' not in x else x)

# data_frames['song_chart.csv'].columns = data_frames['song_chart.csv'].columns.map(lambda x : 'song_'+x if 'song_' not in x else x)
# data_frames['song_pop.csv'].columns = data_frames['song_pop.csv'].columns.map(lambda x : 'song_'+x if 'song_' not in x else x)
# data_frames['album_chart.csv'].columns = data_frames['album_chart.csv'].columns.map(lambda x : 'album_'+x if 'album_' not in x else x)
# data_frames['album_pop.csv'].columns = data_frames['album_pop.csv'].columns.map(lambda x : 'album_'+x if 'album_' not in x else x)
# data_frames['artist_chart.csv'].columns = data_frames['artist_chart.csv'].columns.map(lambda x : 'artist_'+x if 'artist_' not in x else x)
# data_frames['artist_pop.csv'].columns = data_frames['artist_pop.csv'].columns.map(lambda x : 'artist_'+x if 'artist_' not in x else x)

data = pd.merge(data_frames['tracks.csv'], data_frames['songs.csv'], left_on = 'song_id', right_on = 'song_id', how = 'inner', suffixes=['', '_remove'])
data = pd.merge(data, data_frames['albums.csv'], left_on = 'album_id', right_on = 'album_id', how = 'inner', suffixes=['', '_remove'])
data = pd.merge(data, data_frames['releases.csv'], left_on = 'album_id', right_on = 'album_id', how = 'inner', suffixes=['', '_remove'])
data = pd.merge(data, data_frames['artists.csv'], left_on = 'artist_id', right_on = 'artist_id', how = 'inner', suffixes=['', '_remove'])
data = pd.merge(data, data_frames['acoustic_features.csv'], left_on = 'song_id', right_on = 'song_id', how = 'inner', suffixes=['', '_remove'])
# data = pd.merge(data, data_frames['song_chart.csv'], left_on = 'song_id', right_on = 'song_id', how = 'inner', suffixes=['', '_remove'])
# data = pd.merge(data, data_frames['song_pop.csv'], left_on = 'song_id', right_on = 'song_id', how = 'inner', suffixes=['', '_remove'])
# data = pd.merge(data, data_frames['album_chart.csv'], left_on = 'album_id', right_on = 'album_id', how = 'inner', suffixes=['', '_remove'])
# data = pd.merge(data, data_frames['album_pop.csv'], left_on = 'album_id', right_on = 'album_id', how = 'inner', suffixes=['', '_remove'])
# data = pd.merge(data, data_frames['artist_chart.csv'], left_on = 'artist_id', right_on = 'artist_id', how = 'inner', suffixes=['', '_remove'])
# data = pd.merge(data, data_frames['artist_pop.csv'], left_on = 'artist_id', right_on = 'artist_id', how = 'inner', suffixes=['', '_remove'])

## Commented out merges Duplicate rows, which requries ~64GB of memmory to process, TODO: change duplicate rows to be stored as a dictornary to remove the need to duplicate rows

data.drop([i for i in data.columns if 'remove' in i], axis=1, inplace=True)

data.to_csv('test.csv')


# 3. Exploratory Data Analysis

first we start with initial data exploration with use of .head(), .info(), .describe() to gain an understanding of the structure and summary statistics of the dataset

In [ ]:
# Exploration code
print(data.head())

In [ ]:
print(data.info())

In [ ]:
print(data.describe())

## Initial Data Exploration Findings

Our initial exploration of the dataset reveals a rich collection of music tracks, with a total of 11,296 entries across 39 attributes. This extensive dataset allows us to delve into a diverse array of features ranging from song popularity to album characteristics.

Upon inspecting the first few records with `.head()`, we observe a variety of genres and release dates, indicating a dataset that spans a broad temporal range and encompasses a wide spectrum of musical styles.

The data types and completeness assessed by `.info()` suggest that our dataset is mostly complete, but we'll need to address a few missing values to ensure the robustness of our analysis. Furthermore, the numerical summaries provided by `.describe()` offer insights into the central tendencies and dispersion of key variables such as song popularity, which has an average value of approximately 43.49 but varies widely as indicated by the standard deviation.

These initial findings set the stage for a deeper analysis where we will further investigate the factors contributing to song popularity and how these factors have evolved over the time frame covered by our dataset.

In [ ]:
# Figures
df = data
print(df.describe())

# Check for Missing Values
print(df.isnull().sum())

# Distribution of Numeric Features
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
for column in numeric_columns:
    plt.figure(figsize=(10, 4))
    sns.histplot(df[column], kde=True, bins=30)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

# Correlation Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

Descriptions and discoveries...

# Exploratory Data Analysis: Correlation Heatmap Findings
Our exploratory data analysis included a correlation heatmap to identify relationships between different attributes of songs in our dataset. Notable correlations are discussed below:

Album and Song Popularity: There is a strong positive correlation (0.81) between album popularity and song popularity, suggesting that popular albums tend to feature popular songs. This could imply that an album's overall success can positively influence the popularity of its individual tracks.

Song Explicitness and Popularity: Song explicitness has a moderate positive correlation (0.25) with song popularity. This could indicate that songs with explicit content have a tendency to be more popular, which may reflect current trends in music consumption.

Song Features and Popularity: Several song features show moderate positive correlations with song popularity, including speechiness (0.56), and energy (0.13). Speechiness measures the presence of spoken words in a track, while energy captures the intensity and activity. Higher values in these features might correspond to traits that are characteristic of popular music.

Loudness and Energy: A very strong positive correlation (0.68) is observed between loudness and energy, suggesting that louder tracks are often perceived as more energetic. This is consistent with the idea that higher energy in songs is often associated with higher volume levels.

Acousticness and Instrumentalness: Both acousticness and instrumentalness have a strong negative correlation with loudness (-0.39 and -0.39 respectively). This implies that songs with more acoustic and instrumental elements tend to be quieter.

Danceability, Valence, and Tempo: Danceability shows a small positive correlation with valence (0.21) and tempo (0.21), indicating that songs that are more danceable tend to be happier (higher valence) and faster-paced. However, the correlation is not strong enough to draw definitive conclusions, and further analysis might be required.

# 4. Data Pre-Processing
---
Data Pre-Processing is a necessary step in data anaylsis, involving the cleaning and organizing of the raw data given. This data needs to be transformed into a suitable format for its analysis and modelling.This is to improve the quality of the data and make it more suitable for manipulating.

In [ ]:
from scipy import stats
#remove rows with missing values
df.dropna(inplace=True)

#detect outliers using z-score
z_scores = stats.zscore(df[['track_number']])
outliers = (abs(z_scores) < 3).all(axis=1)
df = df[outliers]

In [ ]:
#List of columns to remove
columns_to_remove = [
    'track_number',
    'release_date_precision',
    'song_billboard',
    'album_billboard',
    'album_total_tracks',
    'album_image_url',
    'artist_image_url',
    'song_time_signature'
]

#Makes a copy of the DataFrame
df_cleaned = df.copy()

#Remove specified columns from the copy
df_cleaned.drop(columns=columns_to_remove, inplace=True)

In [ ]:
df_cleaned.rename(columns={
    'song_id': 'Song_ID',
    'album_id': 'Album_ID',
    'release_date': 'Release_Date',
    'song_name': 'Song_Name',
    'song_artists': 'Artists',
    'song_popularity': 'Song_Popularity',
    'song_explicit': 'Explicit',
    'song_type': 'Type',
    'album_name': 'Album',
    'album_artists': 'Album_Artists',
    'album_popularity': 'Album_Popularity',
    'album_type': 'Album_Type',
    'artist_id': 'Artist_ID',
    'artist_name': 'Artist_Name',
    'artist_followers': 'Artist_Followers',
    'artist_popularity': 'Artist_Popularity',
    'artist_type': 'Artist_Type',
    'artist_main_genre': 'Artist_Main_Genre',
    'artist_genres': 'Artist_Genres',
    'song_duration_ms': 'Song_Duration_ms',
    'song_key': 'Key',
    'song_mode': 'Mode',
    'song_acousticness': 'Acousticness',
    'song_danceability': 'Danceability',
    'song_energy': 'Energy',
    'song_instrumentalness': 'Instrumentalness',
    'song_liveness': 'Liveness',
    'song_loudness': 'Loudness',
    'song_speechiness': 'Speechiness',
    'song_valence': 'Valence',
    'song_tempo': 'Tempo'
}, inplace=True)

In [ ]:
print(df_cleaned.head())

##### Dropping the rows with missing values
this is vital as it ensures the integrity of the data while ensuring that the data being worked with is complete as well as consistent.
##### Detecting outliers and removing them
Outliers can significantly affect the performance of statistical analyses,by removing the exremities using the Z-score method, the analysis will not be skewed by outliers in the data ensuring code consistency.
##### Removing certain columns
I removed multiple columns from the dataframe as these columns aren't relevant to the data analysis, the removal of this redundant information reduced the dimensionality of the dataset and focused in on the most relevant features for analysis.It also set up the data to be more readable and managable to a user.
##### Renaming columns
The columns were renamed to increase the interpretability of the dataset, making it easier to understand the meaning of each column as well as making the data more readable

# 5. In-Depth Analysis
---
Initial in depth analysis to confrim heatmap findings in a different way devising which have negetive and positive corrleations towards song popularity only.

In [ ]:
relevant_columns = ['song_popularity', 'song_acousticness', 'song_danceability', 
                    'song_energy', 'song_instrumentalness', 'song_liveness', 
                    'song_loudness', 'song_speechiness', 'song_valence', 'song_tempo']


correlation_matrix = df[relevant_columns].corr()

# Print the correlation of song_popularity with other factors
song_popularity_correlation = correlation_matrix['song_popularity']
print(song_popularity_correlation)

# Filter out non-numeric columns
numeric_columns = df.select_dtypes(include=['number']).columns

# Find the correlation coefficients between song_popularity and all other columns
correlation_with_song_popularity = df[numeric_columns].corr()['song_popularity'].drop('song_popularity')

# Find the variable with the highest correlation coefficient
strongest_effect = correlation_with_song_popularity.abs().idxmax()

# Print the variable with the strongest effect
print("The strongest effect on song_popularity:", strongest_effect , "\n")



# Top 10 features of positive importance
from sklearn.ensemble import RandomForestRegressor
numeric_columns = df.select_dtypes(include='number').columns
X_sampled = df[numeric_columns].drop(['song_popularity'], axis=1)

y_sampled = df['song_popularity']

model = RandomForestRegressor()
model.fit(X_sampled, y_sampled)

feature_importances = pd.Series(model.feature_importances_, index=X_sampled.columns)
top_10_features = feature_importances.nlargest(10)
print("the top 10 things that affect a song positively")
print(top_10_features)




Descriptions and discoveries...

song_acousticness: Correlation coefficient is approximately -0.183. This indicates a weak negative correlation between song popularity and acousticness. As acousticness decreases, song popularity tends to increase slightly.

song_danceability: Correlation coefficient is approximately 0.086. This indicates a very weak positive correlation between song popularity and danceability. There's almost no meaningful relationship between these two variables based on this correlation coefficient.

song_energy: Correlation coefficient is approximately 0.133. This indicates a weak positive correlation between song popularity and energy. As the energy of the song increases, its popularity tends to increase slightly.

song_instrumentalness: Correlation coefficient is approximately -0.082. This indicates a very weak negative correlation between song popularity and instrumentalness. There's almost no meaningful relationship between these two variables based on this correlation coefficient.

song_liveness: Correlation coefficient is approximately -0.039. This indicates a very weak negative correlation between song popularity and liveness. There's almost no meaningful relationship between these two variables based on this correlation coefficient.

song_loudness: Correlation coefficient is approximately 0.333. This indicates a moderate positive correlation between song popularity and loudness. As the loudness of the song increases, its popularity tends to increase.

song_speechiness: Correlation coefficient is approximately 0.139. This indicates a weak positive correlation between song popularity and speechiness. As the speechiness of the song increases, its popularity tends to increase slightly.

song_valence: Correlation coefficient is approximately -0.171. This indicates a weak negative correlation between song popularity and valence. In other words, as the valence (positiveness) of the song decreases, its popularity tends to increase slightly.

song_tempo: Correlation coefficient is approximately 0.011. This indicates almost no correlation between song popularity and tempo. There's almost no meaningful relationship between these two variables based on this correlation coefficient


In [ ]:
# Randomly sample 10% of the data
sampled_df = df.sample(frac=0.1, random_state=42)

sampled_df['release_date'] = pd.to_datetime(sampled_df['release_date'])

# Popularity over time
import numpy as np

# Plot the filtered data 
plt.plot(sampled_df['release_date'], sampled_df['song_popularity'], marker='o', linestyle='', markersize=5)
plt.xlabel('Release Date')
plt.ylabel('Song Popularity')
plt.title('Song Popularity Over Time (Excluding Middle Dates)')
plt.xticks(rotation=45)
plt.show()

# Popularity by genre
plt.figure(figsize=(35, 6))
sns.boxplot(data=sampled_df, x='artist_main_genre', y='song_popularity', palette='Set3')
plt.xlabel('Artist Genre')
plt.ylabel('Song Popularity')
plt.title('Song Popularity by Artist Genre')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sampled_df['release_month'] = sampled_df['release_date'].dt.month

# Define function to map months to seasons
def get_season(month):
    if month in [3, 4, 5]:   
        return 'Spring'
    elif month in [6, 7, 8]:  
        return 'Summer'
    elif month in [9, 10, 11]:  
        return 'Autumn'
    else:                     
        return 'Winter'

# Map months to seasons
sampled_df['season'] = sampled_df['release_month'].apply(get_season)

# Plot average song popularity by season
plt.figure(figsize=(8, 6))
sns.boxplot(data=sampled_df, x='season', y='song_popularity', palette='Set3')
plt.xlabel('Season')
plt.ylabel('Average Song Popularity')
plt.title('Average Song Popularity by Season')
plt.show()



# Assuming 'release_date' is in datetime format
sampled_df['release_date'] = pd.to_datetime(sampled_df['release_date'])

# Extract the month and year from the release date
sampled_df['release_month'] = sampled_df['release_date'].dt.month
sampled_df['release_year'] = sampled_df['release_date'].dt.year

# Group by genre and month, calculate standard deviation of popularity
genre_popularity_std = sampled_df.groupby(['artist_main_genre', 'release_month'])['song_popularity'].std().reset_index()

# Find genres with the highest standard deviation
top_genres = genre_popularity_std.groupby('artist_main_genre')['song_popularity'].mean().nlargest(5).index

# Filter the original dataframe to include only the top genres
filtered_df = sampled_df[sampled_df['artist_main_genre'].isin(top_genres)]

# Plot genre popularity by month for top genres
plt.figure(figsize=(12, 6))
sns.lineplot(data=filtered_df, x='release_month', y='song_popularity', hue='artist_main_genre', palette='Set3', marker='o')
plt.xlabel('Month')
plt.ylabel('Average Song Popularity')
plt.title('Genre Popularity by Month for Top Genres with Most Drastic Changes')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.tight_layout()
plt.show()

Descriptions and discoveries...

Popularity Over Time: The code plots the song popularity over time. Each point on the plot represents a song, with the x-axis representing the release date and the y-axis representing the song popularity. This visualization allows us to observe any trends or patterns in song popularity over time.

Popularity by Genre: The code creates a boxplot showing the distribution of song popularity across different artist genres. This visualization provides insights into how song popularity varies across different genres, allowing us to identify which genres tend to have more popular songs.

Seasonal Trends: The code categorizes release months into seasons (Spring, Summer, Autumn, Winter) and then creates a boxplot showing the average song popularity for each season. This visualization helps us understand whether there are any seasonal patterns in song popularity, such as certain seasons having higher average popularity than others.

Genre Popularity by Month: The code identifies the top genres with the most drastic changes in popularity over different months and then plots the average song popularity by month for these top genres. This visualization allows us to see how the popularity of different genres fluctuates throughout the year and identify any seasonal trends specific to certain genres.

# 6. Communicate Results


### Executive Summary
In our pursuit to unravel the factors influencing song popularity, we performed a comprehensive exploratory data analysis on a rich dataset of music tracks. Our visual and statistical examination has led to revealing insights into how various attributes, from acoustic features to release timing, impact the popularity of songs.

### Trends Over Time
A scatter plot analysis of song popularity over time uncovered a dense cluster of highly popular songs in the most recent decades. This trend could signify a shift in the music industry's distribution strategies or a change in how music is consumed in the digital age. The visualization suggests a more favorable reception to contemporary music, which could be attributed to the proliferation of streaming services and the internet in music distribution.

### Genre-Specific Popularity
Our genre-based analysis, represented through a boxplot, shed light on the heterogeneity of song popularity within various musical styles. It highlighted that certain genres demonstrate a wide range of popularity, potentially due to the eclectic tastes of listeners within those genres. This variability accentuates the uniqueness of musical genres and their unpredictable impact on a song's success.

### Seasonal Influences
Delving into seasonal trends, we found that song popularity exhibits slight variations across different seasons. Our boxplot visualization showed that songs released in Spring and Autumn enjoy marginally higher popularity. This seasonal effect hints at a potential strategic timing for releases by the music industry to maximize song reception.

Correlation with Musical Attributes
Through correlation analysis, we identified key musical attributes that influence song popularity:

- `song_acousticness`: Negative correlation (-0.183027)
- `song_danceability`: Positive correlation (0.086292)
- `song_energy`: Positive correlation (0.133483)
- `song_loudness`: Strong positive correlation (0.333249)
These findings underscore the importance of a song's acoustic profile in determining its popularity, with energetic and louder songs generally being more popular among listeners.

### Monthly Genre Popularity
Investigating further, we observed the monthly variations in genre popularity. Our line plot revealed significant fluctuations, especially in genres such as 'glam rock' and 'detroit hip hop', throughout the year. This fluctuation could be indicative of promotional activities or seasonal listener preferences that influence a genre's popularity.

## Key Takeaways
The strongest predictor of song popularity is the album's popularity, suggesting a strong association between the success of individual tracks and the overall reception of the album they belong to. Other positive contributors to a song's popularity include its danceability and energy, aligning with the contemporary trends favoring dynamic and rhythm-driven music.

## Concluding Remarks
In summary, our exploratory analysis has highlighted the multifaceted nature of song popularity, influenced by a blend of intrinsic musical qualities and external factors such as release timing and album reception. The insights garnered from our study offer a foundation for further inquiry into the evolving landscape of music consumption and the intricate tapestry of factors that resonate with listeners worldwide.

# References
---
The dataset we chose was called MusicOSet,

DataBase Link: https://marianaossilva.github.io/DSW2019/

# Group Reflection

---

As a group we feel that we worked in an organised and concise manner, by collaborating on the work conatantly via discord and github we were able to give our all to completing each task

---
*Contribution Table*
Member | Contribution [% effort]
-|-
Jamie Soan | 100%
Angus Day | 100%
William Heap | 100%
kj | 100%

# Jamie Reflection
---
On the topic of group work, we used git kracken to facilitate a cohesive collaboration of our activities. This came with both good and bad sides as 3 out of the 4 of us had very little experience with it and had to learn how to handle issues when they arose. However working in branches fixed most of these issues and allowed us to code concurrently. On the topic of content within our project, some of our indpeth and exploration of the code overlapped a little which would have been solved with better communication of what exactly we were each implementing within our sections. The overall scope of the analysis we performed on our dataset provides a wide range of different and unique comaprisons and observation of patterns. A more indepth dataset with the statistics of how songs performed over time would have allowed us a much better understanding of the dataset as a whole. Currently with this dataset having only one point of reference date wise for each song, limits us greatly on what we were able to glean from our data. To solve problems like these in future plannning exactly what we want to learn from the data first (acting as our own stakeholders) will help us bet    

# Angus Reflection
---
I believe the coursework went well, this courseowork allowed us to challenge both our programming and team management skills, through this coursework i have learnt quite a large amount around coding with datasets and collaborating around it, going into this coursework we were faced with the challenge of most of us being quite inexperienced working with git kracken however we soon were able to utilise it to streamline the process of collaborating via github.
We were presented with some initial challenges around finding a dataset for the coursework and chose MusicOSet instead of scrubbing the data ourselves. In the process of this courseowork we fell short with time management and from this we will focus on improvement in the future, overall im satisfied with this courseowrk and our overall performance in it, were i to redo it i would allocate more time to ensure a quicker completion

# William Reflection
---
I think the coursework went well. I found the coursework rewarding and I was able to contribute to the group using my Python skills and data analysis techniques. We worked well in the group and openly communicated with each other, using gitkraken to easily edit the notebook along side each other. I encountered some challenges along the way with manipulating the database and with the use of gitkraken as I had hardly used the software previously, but I overcame those struggles as I worked through the coursework. Overall I'm happy with what I accomplished in this coursework though if I were to redo it I would have used a slightly more in-depth dataset to allow for further anaylsis of popularity in songs.

# KJ Reflection
---
Comments... 